In [1]:
from biocypher import BioCypher
from genespectrakg.adapters.genespectra_adapter_individual import (
    GeneSpectraAdapter,
    GeneSpectraAdapterNodeType,
    GeneSpectraAdapterEdgeType,
    GeneSpectraAdapterCellTypeField,
    GeneSpectraAdapterGeneField,
    GeneSpectraAdapterOrthologousGroupField,
    GeneSpectraAdapterSpeciesField,
    GeneSpectraAdapterEdgeField,
)

# Instantiate the BioCypher interface
# You can use `config/biocypher_config.yaml` to configure the framework or
# supply settings via parameters below
bc = BioCypher(
    biocypher_config_path="config/biocypher_config.yaml",
)

# Choose node types to include in the knowledge graph.
# These are defined in the adapter (`adapter.py`).
node_types = [
    GeneSpectraAdapterNodeType.CELL_TYPE,
    GeneSpectraAdapterNodeType.GENE,
    GeneSpectraAdapterNodeType.SPECIES,
    GeneSpectraAdapterNodeType.ORTHOLOGOUS_GROUP,
]

# Choose protein adapter fields to include in the knowledge graph.
# These are defined in the adapter (`adapter.py`).
node_fields = [
    # Proteins
    GeneSpectraAdapterCellTypeField.CELL_TYPE_ID,
    GeneSpectraAdapterCellTypeField.CELL_TYPE_NAME,
    GeneSpectraAdapterCellTypeField.TISSUE_ID,
    GeneSpectraAdapterCellTypeField.TISSUE_NAME,
    GeneSpectraAdapterGeneField.GENE_ID,
    GeneSpectraAdapterGeneField.GENE_NAME,
    GeneSpectraAdapterSpeciesField.SPECIES_ID,
    GeneSpectraAdapterSpeciesField.SPECIES_NAME,
    GeneSpectraAdapterOrthologousGroupField.EGGNOG_DATASET_NAME,
    GeneSpectraAdapterOrthologousGroupField.ORTHOLOGOUS_GROUP_ID,
    GeneSpectraAdapterOrthologousGroupField.EGGNOG_DATASET_ID,
]

edge_types = [
    GeneSpectraAdapterEdgeType.CELL_TYPE_FROM_SPECIES,
    GeneSpectraAdapterEdgeType.GENE_ENHANCED_IN_CELL_TYPE,
    GeneSpectraAdapterEdgeType.GENE_ENRICHED_IN_CELL_TYPE,
    GeneSpectraAdapterEdgeType.GENE_FROM_SPECIES,
    GeneSpectraAdapterEdgeType.GENE_IN_ORTHOLOGOUS_GROUP,
]

edge_fields = [
    GeneSpectraAdapterEdgeField.SPECIFICITY_CATEGORY,
    GeneSpectraAdapterEdgeField.DISTRIBUTION_CATEGORY,
    GeneSpectraAdapterEdgeField.SPECIFICITY_CATEGORY_TYPE,
    GeneSpectraAdapterEdgeField.FRACTION_EXPRESSED,
    GeneSpectraAdapterEdgeField.MAX_EXPRESSION,
    GeneSpectraAdapterEdgeField.MEAN_EXPRESSION,
    GeneSpectraAdapterEdgeField.NUMBER_EXPRESSED,
    GeneSpectraAdapterEdgeField.SPECIFICITY_SCORE,
]

%load_ext autoreload
%autoreload 2


INFO -- This is BioCypher v0.5.19.
INFO -- Logging into `biocypher-log/biocypher-20231130-111257.log`.


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:

# Create a protein adapter instance
adapter = GeneSpectraAdapter(
    node_types=node_types,
    node_fields=node_fields,
    edge_types=edge_types,
    edge_fields=edge_fields,
)

adapter.load_genespectra_data(eggnog_file='data/MTG_eggnog_ensembl_ncbi_genename_mapped_mammalia.csv',
                              cell_ontology_file='data/MTG_cell_type_to_ontology.csv', 
                              genespectra_file='data/all_species_classes_subclass_processed.csv')



INFO -- Loading data.


In [3]:
adapter.eggnog

,ensembl_gene_id,ncbi_gene_name,ncbi_txid,species_scientific_name,eggnog_dataset_name,eggnog_og_id,eggnog_dataset_id
0,ENSG00000121410,A1BG,9606,human,mammalia,8ZMKA,40674
1,ENSG00000175899,A2M,9606,human,mammalia,8ZNU1,40674
2,ENSG00000171428,NAT1,9606,human,mammalia,8Z4IU,40674
3,ENSG00000156006,NAT2,9606,human,mammalia,8Z4IU,40674
4,ENSG00000114771,AADAC,9606,human,mammalia,8ZJEX,40674
...,...,...,...,...,...,...,...
50455,ENSGGOG00000002196,ZNF768,9593,gorilla,mammalia,8ZKSF,40674
50456,ENSGGOG00000025968,VKORC1,9593,gorilla,mammalia,8ZEBQ,40674
50457,ENSGGOG00000016094,SLC25A53,9593,gorilla,mammalia,8ZF2G,40674
50458,ENSGGOG00000004937,SPMIP6,9593,gorilla,mammalia,8ZBQ1,40674


In [4]:

# Create a knowledge graph from the adapter
bc.write_nodes(adapter.get_nodes())



INFO -- Loading ontologies...
INFO -- Instantiating OntologyAdapter class for https://github.com/biolink/biolink-model/raw/v3.2.1/biolink-model.owl.ttl.
INFO -- Creating output directory `/Users/ysong/SOFTWARE/GeneSpectraKG/biocypher-out/20231130111304`.
INFO -- Generating nodes.


get gene nodes from EggNOG
get OG nodes from EggNOG


INFO -- Writing 50460 entries to Gene-part000.csv


get species nodes from EggNOG
get cell type nodes from cell ontology info


INFO -- Writing 16568 entries to OrthologousGroup-part000.csv
INFO -- Writing 5 entries to Species-part000.csv
INFO -- Writing 21 entries to CellType-part000.csv


True

In [5]:
adapter.genespectra_enriched

,cell_ontology_id,cell_type_name,uberon_tissue_id,tissue_name,ensembl_gene_id,ncbi_txid,species_scientific_name,specificity_category,distribution_catehory,specificity_category_type,fraction_expressed,max_expression,mean_expression,n_expressed,specificity_score
5,CL:4023011,lamp5 GABAergic cortical interneuron,UBERON:0002771,middle temporal gyrus,ENSPTRG00000008372,9598,ptroglodytes,cell type enriched,expressed in single,enriched,4.761904761904762,28.1166,1.9869951,1,4.0
6,CL:4030063,L4 intratelencephalic projecting glutamatergic...,UBERON:0002771,middle temporal gyrus,ENSPTRG00000017203,9598,ptroglodytes,group enriched,expressed in over 30%,enriched,61.90476190476191,222.9946,47.659943,13,4.5687554788102425
7,CL:4030065,L6 intratelencephalic projecting glutamatergic...,UBERON:0002771,middle temporal gyrus,ENSPTRG00000017203,9598,ptroglodytes,group enriched,expressed in over 30%,enriched,61.90476190476191,222.9946,47.659943,13,4.5687554788102425
8,CL:4023013,corticothalamic-projecting glutamatergic corti...,UBERON:0002771,middle temporal gyrus,ENSPTRG00000017203,9598,ptroglodytes,group enriched,expressed in over 30%,enriched,61.90476190476191,222.9946,47.659943,13,4.5687554788102425
9,CL:4023011,lamp5 GABAergic cortical interneuron,UBERON:0002771,middle temporal gyrus,ENSPTRG00000017203,9598,ptroglodytes,group enriched,expressed in over 30%,enriched,61.90476190476191,222.9946,47.659943,13,4.5687554788102425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26156,CL:0000129,microglial cell,UBERON:0002771,middle temporal gyrus,ENSCJAG00000067422,9483,cjacchus,group enriched,expressed in less than 30%,enriched,28.57142857142857,225.2656,37.219917,6,14.110625461783023
26157,CL:0002453,oligodendrocyte precursor cell,UBERON:0002771,middle temporal gyrus,ENSCJAG00000067422,9483,cjacchus,group enriched,expressed in less than 30%,enriched,28.57142857142857,225.2656,37.219917,6,14.110625461783023
26158,CL:4023051,vascular leptomeningeal cell,UBERON:0002771,middle temporal gyrus,ENSCJAG00000067422,9483,cjacchus,group enriched,expressed in less than 30%,enriched,28.57142857142857,225.2656,37.219917,6,14.110625461783023
26159,CL:4023051,vascular leptomeningeal cell,UBERON:0002771,middle temporal gyrus,ENSCJAG00000020988,9483,cjacchus,cell type enriched,expressed in less than 30%,enriched,14.285714285714285,294.8587,16.60962,3,4.0


In [6]:

bc.write_edges(adapter.get_edges())


INFO -- Generating edges.


Get cell type from species edges
Get gene from species edges
Get gene from OG edges
Get gene enriched in cell type edges
Get gene enhanced in cell type edges


INFO -- Writing 105 entries to CellTypeFromSpecies-part000.csv
INFO -- Writing 50460 entries to GeneFromSpecies-part000.csv
INFO -- Writing 50460 entries to GeneInOrthologousGroup-part000.csv
INFO -- Writing 15066 entries to GeneEnrichedInCellType-part000.csv
INFO -- Writing 11038 entries to GeneEnhancedInCellType-part000.csv


True

In [7]:

# Write admin import statement
bc.write_import_call()

# Print summary
bc.summary()

INFO -- Writing neo4j import call to `/Users/ysong/SOFTWARE/GeneSpectraKG/biocypher-out/20231130111304/neo4j-admin-import-call.sh`.
INFO -- No duplicate nodes in input.
INFO -- No duplicate edges in input.
INFO -- No missing labels in input.


Showing ontology structure based on https://github.com/biolink/biolink-model/raw/v3.2.1/biolink-model.owl.ttl
entity
├── association
│   ├── cell type from species
│   ├── gene enhanced in cell type
│   ├── gene enriched in cell type
│   ├── gene from species
│   └── gene to gene family association
│       └── gene in orthologous group
└── named thing
    └── biological entity
        ├── gene
        ├── gene family
        │   └── orthologous group
        └── organismal entity
            ├── anatomical entity
            │   └── cell type
            └── cellular organism
                └── species

